# 데이터 환경 설정

In [1]:
import folium
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import decimal
from shapely import wkt
from pyproj import Proj, transform
from haversine import haversine

## 데이터 불러오기

In [2]:
df_ins = pd.read_csv('./data/202011_자연환경조사_곤충_점.csv')
df_rep = pd.read_csv('./data/202011_자연환경조사_양서파충류_점.csv')
df_spi = pd.read_csv('./data/202011_자연환경조사_저서무척추동물_점.csv')
df_bird = pd.read_csv('./data/202011_자연환경조사_조류_점.csv')
df_mam = pd.read_csv('./data/202011_자연환경조사_포유류_점.csv')

* 

In [3]:
df_ins['lati'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_ins['long'] = df_ins['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_rep['lati'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_rep['long'] = df_rep['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_spi['lati'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_spi['long'] = df_spi['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_bird['lati'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_bird['long'] = df_bird['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

df_mam['lati'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[2].replace(')', '')))
df_mam['long'] = df_mam['geom'].apply(lambda x: decimal.Decimal(x.split(' ')[1].replace('(', '')))

In [4]:
df_all = pd.concat([df_ins, df_rep, df_spi, df_bird, df_mam]).copy()
df_all = df_all.reset_index(drop=True)

In [5]:
inProj = Proj(init='epsg:5186')
outProj = Proj(init='epsg:4326')

kr_long, kr_lati = transform(inProj,outProj, list(df_all['lati']), list(df_all['long']))

df_all['kr_lati'] = kr_lati
df_all['kr_long'] = kr_long

In [6]:
#  user = (35.233512, 129.0722714)
user = list(map(decimal.Decimal, input().split(',')))

35.233512, 129.0722714


In [7]:
df_want = df_all[(user[0]-decimal.Decimal(0.5) < df_all['kr_lati']) & (user[0]+decimal.Decimal(0.5) > df_all['kr_lati'])& (user[1]+decimal.Decimal(0.5) > df_all['kr_long'])& (user[1]+decimal.Decimal(0.5) > df_all['kr_long'])].reset_index(drop=True)

In [37]:
want_list = []
for i in df_want.index:
    if haversine(user, (df_want['kr_lati'][i], df_want['kr_long'][i]), unit = 'km') <= 3:
        want_list.append(i)
        
df_find = df_want.loc[want_list].copy()

In [41]:
df_find.groupby('spcs_lcnm').count().sort_values(by = 'spce_id', ascending = False)

,spce_id,examin_realm_se_code_nm,spcs_scncenm,examin_year,examin_begin_de,examin_end_de,geom,lati,long,kr_lati,kr_long
spcs_lcnm,,,,,,,,,,,
고라니,40,40,40,40,40,40,40,40,40,40,40
오소리,22,22,22,22,22,22,22,22,22,22,22
꾀꼬리,15,15,15,15,15,15,15,15,15,15,15
두더지,12,12,12,12,12,12,12,12,12,12,12
멧토끼,8,8,8,8,8,8,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...
딸기잎벌레,1,1,1,1,1,1,1,1,1,1,1
등검은말벌,1,1,1,1,1,1,1,1,1,1,1
두줄박이잎벌레,1,1,1,1,1,1,1,1,1,1,1


In [45]:
zoo_map = folium.Map(location=[user[0], user[1]], zoom_start = 14)

tooltip = '나를 눌러줘!'

for i in want_list:
    if df_want.iloc[i]['examin_realm_se_code_nm']=='곤충':
        folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], 
                            popup = '나는 곤충이야!<br> <b>'+df_want.iloc[i]['spcs_lcnm']+'</b>', tooltip=tooltip, 
                            radius=4, color='yellow').add_to(zoo_map)
    
    elif df_want.iloc[i]['examin_realm_se_code_nm']=='양서파충류':
        folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], 
                            popup = '나는 양서파충류야!<br> <b>'+df_want.iloc[i]['spcs_lcnm']+'</b>', tooltip=tooltip, 
                            radius=4, color='blue').add_to(zoo_map)
        
    elif df_want.iloc[i]['examin_realm_se_code_nm']=='저서무척추동물':
        folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], 
                            popup = '나는 저서무척추동물이야!<br> <b>'+df_want.iloc[i]['spcs_lcnm']+'</b>', tooltip=tooltip, 
                            radius=4, color='red').add_to(zoo_map)
        
    elif df_want.iloc[i]['examin_realm_se_code_nm']=='조류':
        folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], 
                            popup = '나는 조류야!<br> <b>'+df_want.iloc[i]['spcs_lcnm']+'</b>', tooltip=tooltip, 
                            radius=4, color='ivory').add_to(zoo_map)
        
    elif df_want.iloc[i]['examin_realm_se_code_nm']=='포유류':
        if df_want.iloc[i]['spcs_lcnm']=='고라니':
            folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], 
                            popup = '나는 포유류야!<br> <b>'+df_want.iloc[i]['spcs_lcnm']+' <a href="https://ko.wikipedia.org/wiki/%EA%B3%A0%EB%9D%BC%EB%8B%88">더 알아보기</a></b>'
                                +'<img src="./data/고라니.jpg" width="200" height="200" alt="My Image">'
                                ,tooltip=tooltip, 
                            radius=4, color='green').add_to(zoo_map)
        
        else :
            folium.CircleMarker(location=[df_want['kr_lati'][i], df_want['kr_long'][i]], 
                            popup = '나는 포유류야!<br> <b>'+df_want.iloc[i]['spcs_lcnm']+'</b>', tooltip=tooltip, 
                            radius=4, color='green').add_to(zoo_map)

zoo_map

In [46]:
zoo_map.save('map.html')